In [ ]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona 

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

# BDTOPO

In [ ]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")

# on crée la var qui nous servira de filtre sur les autres db
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)

In [ ]:
# 1. Bâtiments
bats = gpd.read_file(GPKG, layer="batiment")
bats = bats.to_crs(4326)
#TODO: confirm use of intersect (instead of within) with Luc or else
bats_st_denis = gpd.sjoin(bats, saint_denis, how='inner', predicate='intersects')

# 2. Zones d'éducation
bdedu = gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet")
bdedu = bdedu.to_crs(4326)
bdedu = bdedu[bdedu["categorie"] == "Science et enseignement"]
bdedu_st_denis = gpd.sjoin(bdedu, saint_denis, how='inner', predicate='intersects')

In [ ]:
# 1. Plot des bâtiments
fig1, ax1 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax1, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bats_st_denis.plot(ax=ax1, color='blue', alpha=0.5, label='Bâtiments')
cx.add_basemap(ax1, crs=bats_st_denis.crs, zoom=14)
ax1.set_title(f"Bâtiments à Saint-Denis\n{len(bats_st_denis)} bâtiments", pad=20)
plt.show()

In [ ]:
batiment_en_zone_scolaire = gpd.sjoin(bats_st_denis.drop(columns=["index_right"]), bdedu_st_denis.drop(columns=["index_right"]), how='inner', predicate='intersects')
fig1, ax1 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax1, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
batiment_en_zone_scolaire.plot(ax=ax1, color='blue', alpha=0.5, label='Bâtiments')
cx.add_basemap(ax1, crs=bats_st_denis.crs, zoom=14)
ax1.set_title(f"Bâtiments à Saint-Denis\n{len(bats_st_denis)} bâtiments", pad=20)
plt.show()

In [ ]:
# 2. Plot des zones d'éducation
fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bdedu_st_denis.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
plt.show()

# Annuaires des établissements scolaires 

In [ ]:
annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
annuaire_st_denis = gpd.sjoin(annuaire, saint_denis, how='inner', predicate='within')

In [ ]:
# 2. Plot des zones d'éducation
fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bdedu_st_denis.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
annuaire_st_denis.plot(ax=ax2, color='green', alpha=0.5, label='Établissements scolaires')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
plt.show()

# Immeubles protégés

In [ ]:
immeubles_proteges = gpd.read_file(DATA_FOLDER / "liste-des-immeubles-proteges-au-titre-des-monuments-historiques.geojson").to_crs(4326)
immeubles_proteges = immeubles_proteges[immeubles_proteges.departement_format_numerique == "93"]
print(len(immeubles_proteges))
immeubles_proteges_st_denis = gpd.sjoin(immeubles_proteges, saint_denis, how='inner', predicate='within')

In [ ]:
# 2. Plot des zones d'éducation
fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bdedu_st_denis.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
annuaire_st_denis.plot(ax=ax2, color='green', alpha=0.5, label='Établissements scolaires')
immeubles_proteges_st_denis.plot(ax=ax2, color='red', alpha=0.5, label='Immeubles protégés')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
plt.show()

## Zones protégées dans les zones d'établissements scolaires

In [ ]:
immeubles_proteges_st_denis_utm = immeubles_proteges_st_denis.to_crs(epsg=32631)
immeubles_proteges_buffer_500m  = immeubles_proteges_st_denis.buffer(0.005, cap_style="round").to_crs(epsg=4326)

fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bdedu_st_denis.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
annuaire_st_denis.plot(ax=ax2, color='green', alpha=0.5, label='Établissements scolaires')
immeubles_proteges_st_denis.plot(ax=ax2, color='red', alpha=0.5, label='Immeubles protégés')
immeubles_proteges_buffer_500m.plot(ax=ax2, color='red', alpha=0.2, edgecolor='red', label='Buffer 500m')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
plt.show()

In [ ]:
# create a column True if a building is in the buffer
bdedu_st_denis["en_zone_classe"] = bdedu_st_denis.intersects(immeubles_proteges_buffer_500m.unary_union)
bdedu_st_denis.head()

In [ ]:
print("Nombre de zone d'éducation situées dans une zone classée:", bdedu_st_denis["en_zone_classe"].sum(), "/", len(bdedu_st_denis))
bdedu_st_denis["en_zone_classe"].value_counts()

In [ ]:
immeubles_proteges_st_denis_utm = immeubles_proteges_st_denis.to_crs(epsg=32631)

immeubles_proteges_buffer_500m = immeubles_proteges_st_denis.buffer(0.005, cap_style="round").to_crs(epsg=4326)
immeubles_proteges_buffer_500m = gpd.GeoDataFrame(geometry=immeubles_proteges_buffer_500m, crs=4326)

bdedu_st_denis = bdedu_st_denis.drop(columns=["index_right"])

bdedu_st_denis_in_buffer = bdedu_st_denis.sjoin(immeubles_proteges_buffer_500m, how='inner', predicate='intersects')
bdedu_st_denis_in_buffer.drop_duplicates()

fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bdedu_st_denis_in_buffer.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
annuaire_st_denis.plot(ax=ax2, color='green', alpha=0.5, label='Établissements scolaires')
immeubles_proteges_st_denis.plot(ax=ax2, color='red', alpha=0.5, label='Immeubles protégés')
immeubles_proteges_buffer_500m.plot(ax=ax2, color='red', alpha=0.2, edgecolor='red', label='Buffer 500m')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
plt.show()

# Analyse des batiments appartenants à des zones éducatives 

In [ ]:
# immeubles_proteges_buffer_500m = immeubles_proteges_st_denis.drop(columns=["index_right"])
batiment_en_zone_scolaire = batiment_en_zone_scolaire.drop(columns=["index_right"])

batiment_en_zone_scolaire = batiment_en_zone_scolaire.sjoin(immeubles_proteges_buffer_500m, how='inner', predicate='intersects')
batiment_en_zone_scolaire = batiment_en_zone_scolaire.drop_duplicates()

fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
batiment_en_zone_scolaire.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
annuaire_st_denis.plot(ax=ax2, color='green', alpha=0.5, label='Établissements scolaires')
immeubles_proteges_st_denis.plot(ax=ax2, color='red', alpha=0.5, label='Immeubles protégés')
immeubles_proteges_buffer_500m.plot(ax=ax2, color='red', alpha=0.2, edgecolor='red', label='Buffer 500m')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
plt.show()

In [ ]:
batiment_en_zone_scolaire["en_zone_classe"] = batiment_en_zone_scolaire.intersects(immeubles_proteges_buffer_500m.unary_union)
batiment_en_zone_scolaire = batiment_en_zone_scolaire.drop_duplicates()
batiment_en_zone_scolaire.head()